In [ ]:
!pip install monai

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
from dotenv import load_dotenv

import os
load_dotenv() # This loads the .env file

from monai.apps import download_url, download_and_extract


# Get active session (automatically available in Snowflake Notebooks)
connection_parameters = dict(
    account = 'sfsenorthamerica-cguzman_aws_us_west_2',
    user = 'carlos.guzman@snowflake.com',
    password = os.getenv('SNOWFLAKE_PASSWORD'),
    role = 'sysadmin',
    warehouse = 'compute_wh',
    database = 'sf_clinical_db',
    schema = 'utils'
 )
 
try:
    session = get_active_session()
except:
    session = Session.builder.configs(connection_parameters).create()




In [ ]:
session.sql("select current_user()").collect()

In [ ]:
import tempfile

directory = '/tmp'
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

In [ ]:
resource = "https://zenodo.org/record/3835682/files/training.zip"

compressed_file = os.path.join(root_dir, "paired_ct_lung.zip")
data_dir = os.path.join(root_dir, "paired_ct_lung")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir)
    os.rename(os.path.join(root_dir, "training"), data_dir)

In [ ]:
session.use_database("SF_CLINICAL_DB")
session.use_schema("UTILS")


session.file.put(f"{data_dir}/lungMasks/*exp.nii.gz", "@monai_medical_images_stg/lungMasksExp", overwrite=True)
session.file.put(f"{data_dir}/lungMasks/*insp.nii.gz", "@monai_medical_images_stg/lungMasksInsp", overwrite=True)
session.file.put(f"{data_dir}/scans/*exp.nii.gz", "@monai_medical_images_stg/scansExp", overwrite=True)
session.file.put(f"{data_dir}/scans/*insp.nii.gz", "@monai_medical_images_stg/scansInsp", overwrite=True)

